<a href="https://colab.research.google.com/github/Hrishkesh/Invoice-/blob/master/Invoice_new.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!pip install --user tensorflow-gpu==2.0.0

     |████████████████████████████████| 380.8MB 44kB/s 
     |████████████████████████████████| 450kB 39.1MB/s 
     |████████████████████████████████| 3.8MB 38.4MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp36-none-any.whl size=7540 sha256=7cf82dd750906a095bdae38dafba8af2e0573711c862dce14243fe325b5adbc4
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
ERROR: tensorflow 2.2.0 has requirement gast==0.3.3, but you'll have gast 0.2.2 which is incompatible.
ERROR: tensorflow 2.2.0 has requirement tensorboard<2.3.0,>=2.2.0, but you'll have tensorboard 2.0.2 which is incompatible.
ERROR: tensorflow 2.2.0 has requirement tensorflow-estimator<2.3.0,>=2.2.0, but you'll have tensorflow-estimator 2.0.1 which is incompatible.
ERROR: tensorflow-probability 0.10.0 has requirement gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.
  Consider adding this directory to PATH or, if you prefer to suppress

In [6]:
!pip install tensorboard

In [7]:
!pip install --upgrade wandb
!wandb login 47ae077cee2c514d2b004ca904e0a365185a06c1

     |████████████████████████████████| 1.4MB 3.5MB/s 
     |████████████████████████████████| 102kB 9.7MB/s 
     |████████████████████████████████| 460kB 18.8MB/s 
     |████████████████████████████████| 102kB 9.9MB/s 
     |████████████████████████████████| 112kB 25.8MB/s 
     |████████████████████████████████| 71kB 8.6MB/s 
     |████████████████████████████████| 71kB 8.7MB/s 
  Created wheel for watchdog: filename=watchdog-0.10.2-cp36-none-any.whl size=73605 sha256=dadae74ad7464de63e8662fe2ece47bf02d68daf9a148bc0afe6483c2ef2897c
  Stored in directory: /root/.cache/pip/wheels/bc/ed/6c/028dea90d31b359cd2a7c8b0da4db80e41d24a59614154072e
  Created wheel for gql: filename=gql-0.2.0-cp36-none-any.whl size=7630 sha256=6928e3bde87798ad7e3db3b1fec8cd9a1d1390683d720546359c77e7b9df138a
  Stored in directory: /root/.cache/pip/wheels/ce/0e/7b/58a8a5268655b3ad74feef5aa97946f0addafb3cbb6bd2da23
  Created wheel for subprocess32: filename=subprocess32-3.5.4-cp36-none-any.whl size=6489 sha256=7685

In [8]:
import tensorflow as tf
tf.__version__

'2.2.0'

In [9]:
!wget https://www.dropbox.com/s/u2dccpjgs3ynm8x/invoice_eng.tar.gz

--2020-05-31 11:37:47--  https://www.dropbox.com/s/u2dccpjgs3ynm8x/invoice_eng.tar.gz
Resolving www.dropbox.com (www.dropbox.com)... 162.125.3.1, 2620:100:6018:1::a27d:301
Connecting to www.dropbox.com (www.dropbox.com)|162.125.3.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/u2dccpjgs3ynm8x/invoice_eng.tar.gz [following]
--2020-05-31 11:37:48--  https://www.dropbox.com/s/raw/u2dccpjgs3ynm8x/invoice_eng.tar.gz
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc29c3f67bb76309449117d5f30d.dl.dropboxusercontent.com/cd/0/inline/A4wrPAUhOi3ZQQghYYWAP5lx3_orad4mVFnfCfo41gP73hg_oTPV8Bf2lOil4w0zQvn64_rMOdmbAQNHVGdse3T5ABH-DmwKULKDKjf-rVIR56QRStz-YftD71DqA3P1juo/file# [following]
--2020-05-31 11:37:48--  https://uc29c3f67bb76309449117d5f30d.dl.dropboxusercontent.com/cd/0/inline/A4wrPAUhOi3ZQQghYYWAP5lx3_orad4mVFnfCfo41gP73hg_oTPV8Bf2lOil4w0zQvn64_rMOdmbAQNHVGdse3T5ABH-DmwK

In [10]:
!ls

invoice_eng.tar.gz  ngrok  ngrok-stable-linux-amd64.zip  sample_data  wandb


In [0]:
!gunzip invoice_eng.tar.gz

In [12]:
!ls

invoice_eng.tar  ngrok	ngrok-stable-linux-amd64.zip  sample_data  wandb


In [13]:
!tar -xvf invoice_eng.tar

invoice_eng/
invoice_eng/outdir_np_chargrid_1h/
invoice_eng/outdir_np_chargrid_1h/X51006555806.npy
invoice_eng/outdir_np_chargrid_1h/X51007339152.npy
invoice_eng/outdir_np_chargrid_1h/X51007339126.npy
invoice_eng/outdir_np_chargrid_1h/X51006556656.npy
invoice_eng/outdir_np_chargrid_1h/X51006619563.npy
invoice_eng/outdir_np_chargrid_1h/X51007579726.npy
invoice_eng/outdir_np_chargrid_1h/X51005757290.npy
invoice_eng/outdir_np_chargrid_1h/X51006913031.npy
invoice_eng/outdir_np_chargrid_1h/X51008142032.npy
invoice_eng/outdir_np_chargrid_1h/X51007339139.npy
invoice_eng/outdir_np_chargrid_1h/X51005568914.npy
invoice_eng/outdir_np_chargrid_1h/X51007846307.npy
invoice_eng/outdir_np_chargrid_1h/X51007846305.npy
invoice_eng/outdir_np_chargrid_1h/X51005568898.npy
invoice_eng/outdir_np_chargrid_1h/X51007103675.npy
invoice_eng/outdir_np_chargrid_1h/X51005746140.npy
invoice_eng/outdir_np_chargrid_1h/X51005676541.npy
invoice_eng/outdir_np_chargrid_1h/X51006620189.npy
invoice_eng/outdir_np_chargrid_1h/

In [0]:
!rm -rf invoice_eng.tar

In [15]:
!ls 

invoice_eng  ngrok  ngrok-stable-linux-amd64.zip  sample_data  wandb


In [16]:
!git clone https://github.com/Hrishkesh/Invoice-.git

Cloning into 'Invoice-'...
remote: Enumerating objects: 52, done.
remote: Counting objects: 100% (52/52), done.
remote: Compressing objects: 100% (42/42), done.
remote: Total 52 (delta 24), reused 23 (delta 8), pack-reused 0
Unpacking objects: 100% (52/52), done.


In [17]:
!ls Invoice-/

Invoice_new.ipynb  network.py  preprocessing.py  README.md


In [18]:
import os
from getpass import getpass
user = getpass('GitHub user')
password = getpass('GitHub password')
os.environ['GITHUB_AUTH'] = user + ':' + password
os.environ['GITHUB_USER'] = user
os.environ['GITHUB_PASSWORD'] = user

GitHub user··········
GitHub password··········


In [0]:
!git config --global user.email "hrishkesh.pattepu@gmail.com"
!git config --global user.name "Hrishkesh"

In [0]:
!cd Invoice- && git remote set-url origin https://$GITHUB_AUTH@github.com/$GITHUB_USER/Invoice-.git

In [21]:
!cd Invoice- && git pull

Already up to date.


In [0]:
!cd Invoice- && git add .

In [23]:
!cd Invoice- && git commit -m "training"

On branch master
Your branch is up to date with 'origin/master'.

nothing to commit, working tree clean


In [24]:
!cd Invoice- && git push

Everything up-to-date


In [2]:
!pip install colab_ssh --upgrade

In [4]:
from colab_ssh import launch_ssh, init_git
from getpass import getpass
ngrokToken = '1cfbiKCtHgohcxmNMIXe34CpbET_7GibpvfZdp55n7ordbjoT'
password = getpass('Password of your choice')
launch_ssh(ngrokToken,password)

Password of your choice··········
Successfully running 0.tcp.ngrok.io:18115
[Optional] You can also connect with VSCode SSH Remote extension using this configuration:

	  Host google_colab_ssh
		  HostName 0.tcp.ngrok.io
		  User root
		  Port 18115
	  


In [0]:
!python Invoice-/network.py

wandb: Tracking run with wandb version 0.8.36
wandb: Run data is saved locally in wandb/run-20200527_104748-1svwbwkx
wandb: Syncing run gallant-leaf-2
wandb: ⭐️ View project at https://app.wandb.ai/hrishkeshpattepu/invoice-new
wandb: 🚀 View run at https://app.wandb.ai/hrishkeshpattepu/invoice-new/runs/1svwbwkx
wandb: Run `wandb off` to turn off syncing.

2020-05-27 10:47:50.261925: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
2020-05-27 10:47:50.273099: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2300000000 Hz
2020-05-27 10:47:50.273837: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x2085480 executing computations on platform Host. Devices:
2020-05-27 10:47:50.273877: I tensorflow/compiler/xla/service/service.cc:175]   StreamExecutor device (0): Host, Default Version
2020-05-27 10:47:50.302935: I tensorflow/stream_executor/platform/default/d